# N-Queen Problem using Evolutionary Algorithms
## Kiarash Mokhtari - 9830032

In [48]:
import random
import time

### 1. First we define Initialize Population function which randomly generates individuals.

In [49]:
def initialize_population(population_size: int, board_size: int) -> list:
    population = []
    for _ in range(population_size):
        individual = [random.randint(0, board_size-1) for _ in range(board_size)]
        population.append(individual)

    return population

In [50]:
# Inorder to test the function we can run below code:
def print_matrix(matrix):
    for row in matrix:
        print(row)
        
population_size = 10
board_size = 8

sample_population = initialize_population(population_size, board_size)
print_matrix(sample_population)

[3, 3, 3, 5, 1, 2, 1, 0]
[2, 5, 7, 7, 5, 7, 6, 3]
[0, 2, 6, 7, 2, 6, 5, 4]
[0, 0, 6, 3, 0, 3, 2, 5]
[0, 7, 4, 2, 5, 0, 6, 1]
[1, 4, 6, 3, 7, 6, 3, 3]
[3, 1, 0, 5, 6, 1, 7, 6]
[0, 0, 2, 0, 1, 3, 2, 0]
[5, 5, 1, 4, 1, 1, 6, 6]
[4, 7, 4, 5, 4, 5, 5, 2]


### 2. we define Evaluate Fitness fucntion which evaluates the fitness of each individual in the population.

In [51]:
def evaluate_fitness(individual):
    conflicts = 0
    size = len(individual)

    for i in range(size):
        for j in range(i+1, size):
            if individual[i] == individual[j] or abs(individual[i] - individual[j]) == abs(i - j):
                conflicts += 1

    fitness = 1 / (conflicts + 1)

    return fitness

In [52]:
# Inorder to test the function we can run below code:
for individual in sample_population:
    print(evaluate_fitness(individual))

0.09090909090909091
0.1
0.125
0.09090909090909091
0.2
0.125
0.16666666666666666
0.09090909090909091
0.1111111111111111
0.08333333333333333


### 3. we define Selection fucntion which selects individuals from the population based on their fitness values. .

In [53]:
# 1 approach
def tournament_selection(population, tournament_size):
    selected = []
    
    for _ in range(len(population)):
        sub_population = random.sample(population, tournament_size)
        winner = max(sub_population, key=evaluate_fitness)
        selected.append(winner)
    
    return selected

### 4. we define Crossover fucntion which performs crossover or recombination between selected individuals to create offspring.


In [54]:
def crossover(parent1, parent2):
    size = len(parent1)
    crossover_point = random.randint(1, size - 1)

    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return crossover_point, child1, child2

In [55]:
# Inorder to test the function we can run below code:
parent1 = [0, 3, 6, 4, 7, 1, 5, 2]
parent2 = [4, 1, 3, 6, 2, 0, 7, 5]

print("Children before Crossover:")
print(parent1)
print(parent2)
print("========================================================")

crossover_point, child1, child2 = crossover(parent1, parent2)
print("Children after Crossover:")
print(child1)
print(child2)
print("Crossover Point: ", crossover_point)

Children before Crossover:
[0, 3, 6, 4, 7, 1, 5, 2]
[4, 1, 3, 6, 2, 0, 7, 5]
Children after Crossover:
[0, 1, 3, 6, 2, 0, 7, 5]
[4, 3, 6, 4, 7, 1, 5, 2]
Crossover Point:  1


### 5. we define Mutation fucntion which adds diversity to the population and helps explore different regions of the solution space.

In [56]:
def mutation(individual, mutation_rate):
    size = len(individual)
    mutated_individual = individual.copy()

    for i in range(size):
        if random.random() < mutation_rate:
            new_position = random.randint(0, size - 1)
            mutated_individual[i] = new_position

    return mutated_individual

In [57]:
# Inorder to test the function we can run below code:
individual = [0, 3, 6, 4, 7, 1, 5, 2]
mutation_rate = 0.1

mutated_individual = mutation(individual, mutation_rate)
print(mutated_individual)

[0, 3, 6, 4, 7, 1, 5, 2]


### 6. we define Replacement fucntion which selects individuals from the parent population and the offspring population to form the next generation.

In [58]:
def replacement(parent_population, offspring_population, population_size):
    combined_population = parent_population + offspring_population

    combined_population.sort(key=evaluate_fitness, reverse=True)

    next_generation = combined_population[:population_size]

    return next_generation

### 7. we define Termination fucntion which determines when to stop the algorithm.

In [59]:
def termination_condition(population, max_fitness):
    max_population_fitness = max(evaluate_fitness(individual) for individual in population)
    return max_population_fitness == max_fitness

### 8. Main function tournament_selection function

In [60]:
def solve_n_queen(population_size, board_size, max_generations, mutation_rate):
    population = initialize_population(population_size, board_size)
    current_generation = 0

    while not termination_condition(population, 1.0):
        parent_population = tournament_selection(population, 3)
        offspring_population = []

        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(parent_population, 2)
            crossover_point, child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, mutation_rate)
            mutated_child2 = mutation(child2, mutation_rate)
            offspring_population.extend([mutated_child1, mutated_child2])

        population = replacement(parent_population, offspring_population, population_size)
        current_generation += 1

    best_individual = max(population, key=evaluate_fitness)
    best_fitness = evaluate_fitness(best_individual)
    return best_individual, best_fitness

In [62]:
def print_individual(individual):
    board_size = len(individual)

    for row in range(board_size):
        line = ""
        for col in range(board_size):
            if individual[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line.strip())
        
# Example for 8-Queen
population_size = 1000
board_size = 8
max_generations = 1000
mutation_rate = 0.1

start_time = time.time()
best_individual, best_fitness = solve_n_queen(population_size, board_size, max_generations, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("8-Queen Problem:")
print("Best Individual:", best_individual)
print_individual(best_individual)
print("Best Fitness:", best_fitness)
print("Execution Time:", execution_time, "seconds")

print("\n")

# Example for 16-Queen
population_size = 200
board_size = 16
max_generations = 2000
mutation_rate = 0.5

start_time = time.time()
best_individual, best_fitness = solve_n_queen(population_size, board_size, max_generations, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("16-Queen Problem:")
print("Best Individual:", best_individual)
print_individual(best_individual)
print("Best Fitness:", best_fitness)
print("Execution Time:", execution_time, "seconds")

8-Queen Problem:
Best Individual: [3, 1, 7, 5, 0, 2, 4, 6]
- - - Q - - - -
- Q - - - - - -
- - - - - - - Q
- - - - - Q - -
Q - - - - - - -
- - Q - - - - -
- - - - Q - - -
- - - - - - Q -
Best Fitness: 1.0
Execution Time: 0.14599967002868652 seconds


16-Queen Problem:
Best Individual: [0, 13, 6, 4, 14, 11, 5, 3, 1, 12, 15, 9, 7, 10, 8, 2]
Q - - - - - - - - - - - - - - -
- - - - - - - - - - - - - Q - -
- - - - - - Q - - - - - - - - -
- - - - Q - - - - - - - - - - -
- - - - - - - - - - - - - - Q -
- - - - - - - - - - - Q - - - -
- - - - - Q - - - - - - - - - -
- - - Q - - - - - - - - - - - -
- Q - - - - - - - - - - - - - -
- - - - - - - - - - - - Q - - -
- - - - - - - - - - - - - - - Q
- - - - - - - - - Q - - - - - -
- - - - - - - Q - - - - - - - -
- - - - - - - - - - Q - - - - -
- - - - - - - - Q - - - - - - -
- - Q - - - - - - - - - - - - -
Best Fitness: 1.0
Execution Time: 1627.0384032726288 seconds
